In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from pydub import AudioSegment
import os
import random

[paper 1](https://arxiv.org/pdf/2007.11154)
[fuente 1](https://dolby.io/blog/beginners-guide-to-visualizing-audio-as-a-spectogram-in-python/)
[fuente 2](https://wandb.ai/mostafaibrahim17/ml-articles/reports/An-Introduction-to-Audio-Classification-with-Keras--Vmlldzo0MDQzNDUy)

In [2]:
train = pd.read_csv('/kaggle/input/udp-dl-trabajo-3-covid-19/train.csv')
X_train = train.drop(columns=["covid"])
y_train = train["covid"]
print(X_train)
print(y_train)

                                      uuid
0     b43f9802-2907-47b1-a0a9-2aaa0bdc9e7b
1     9ff3f723-e33b-46a2-84f6-91e3efeb6eaa
2     a429e6d7-fc50-4d99-a49b-748a6d9be187
3     5a48be84-91c7-4200-8e13-acc90e8e68d9
4     b062afcd-e4ee-4d70-9206-7614bfe7c840
...                                    ...
6578  513a5d84-fa87-49f2-8eec-b058c4b57885
6579  f107919f-1a16-48e8-a3a2-2d683f151a6d
6580  9de87976-0594-4251-99a7-5dfbd6da22f9
6581  c5a34881-c434-4883-bfc2-6846ab23bf4a
6582  d617edc2-6117-427d-92af-c2dae06df70c

[6583 rows x 1 columns]
0       0
1       0
2       0
3       0
4       0
       ..
6578    0
6579    0
6580    0
6581    0
6582    0
Name: covid, Length: 6583, dtype: int64


In [3]:
# Create a directory to save the spectrogram images
output_dir = '/kaggle/working/spectrograms/'
os.makedirs(output_dir, exist_ok=True)

In [4]:
# Count the number of COVID samples
covid_samples = train[train['covid'] == 1]
non_covid_samples = train[train['covid'] == 0]

n_covid = len(covid_samples)
n_total = int(2822 / 0.2)  # Calculamos el total de datos
n_train = int(n_total * 0.7)  # 70% del total
n_test = 2822  # 20% del total

# Select samples for testing
n_non_covid_test = n_test - n_covid  # El resto para no-covid en prueba
non_covid_test_samples = non_covid_samples.sample(n=n_non_covid_test, random_state=42)
remaining_non_covid_samples = non_covid_samples.drop(non_covid_test_samples.index)

# Select samples for training
n_non_covid_train = min(n_train - n_covid, len(remaining_non_covid_samples))
non_covid_train_samples = remaining_non_covid_samples.sample(n=n_non_covid_train, random_state=42)

# Create the balanced training and testing datasets
train_samples = pd.concat([covid_samples, non_covid_train_samples])
test_samples = pd.concat([non_covid_test_samples])

# Create directories to save the spectrogram images
train_output_dir = '/kaggle/working/train_spectrograms/'
test_output_dir = '/kaggle/working/test_spectrograms/'
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)

In [5]:
def convert_webm_to_wav(webm_path, wav_path):
    audio = AudioSegment.from_file(webm_path, format='webm')
    audio.export(wav_path, format='wav')

In [6]:
def normalize_audio(aud):
    aud = aud / np.max(np.abs(aud))  # Normalize to -1 to 1
    return aud


In [7]:

# Lists to keep track of processed images and their labels
train_labels = []
test_labels = []

In [8]:
def generate_spectrograms(samples, output_dir, labels_list):
    count_a = 0
    count_b = 0
    for index, row in samples.iterrows():
        print(f"INDEX {index} - Processing UUID: {row['uuid']}, COVID: {row['covid']}")
        
        webm_path = f'/kaggle/input/udp-dl-trabajo-3-covid-19/audio/{row["uuid"]}.webm'
        wav_path = f'/kaggle/working/{row["uuid"]}.wav'
        
        # Check if the .webm file exists, if not, try .wav
        if os.path.exists(webm_path):
            convert_webm_to_wav(webm_path, wav_path)
        elif os.path.exists(webm_path.replace('.webm', '.wav')):
            wav_path = webm_path.replace('.webm', '.wav')
        else:
            print(f"Audio file for UUID {row['uuid']} not found.")
            count_b += 1
            continue
        
        # Read the WAV file
        Fs, aud = wavfile.read(wav_path)
        
        # Normalize the audio data
        aud = normalize_audio(aud)
        
        # Ensure the audio data is mono
        if len(aud.shape) > 1:
            aud = aud.mean(axis=1)
        
        # Generate the spectrogram with adjusted parameters
        plt.figure(figsize=(10, 4))
        powerSpectrum, frequenciesFound, time, imageAxis = plt.specgram(aud, NFFT=1024, Fs=Fs, noverlap=512)
        
        # Replace zeros with a small value to avoid log(0)
        powerSpectrum[powerSpectrum == 0] = np.finfo(float).eps
        
        # Save the spectrogram as an image
        output_path = os.path.join(output_dir, f'{row["uuid"]}_covid_{row["covid"]}.png')
        plt.axis('off')
        plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close()
        
        # Append the label to the list
        labels_list.append(row['covid'])
        count_a += 1
        
        # Optionally, remove the temporary WAV file if it was converted
        if os.path.exists(wav_path):
            os.remove(wav_path)
    
    print(f"Processed: {count_a}, Skipped: {count_b}")

In [9]:
# Generate spectrograms for training and testing datasets
#generate_spectrograms(train_samples, train_output_dir, train_labels)
generate_spectrograms(test_samples, test_output_dir, test_labels)

INDEX 3598 - Processing UUID: 4153e3e7-08d9-4841-b30c-1b772b7d4c27, COVID: 0


/opt/conda/lib/python3.10/site-packages/matplotlib/axes/_axes.py:7774: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


INDEX 324 - Processing UUID: 82793616-0787-4198-b2f0-991ea3df25e9, COVID: 0


/opt/conda/lib/python3.10/site-packages/matplotlib/axes/_axes.py:7774: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


INDEX 381 - Processing UUID: 220ee992-c9d0-43e6-b709-3e5884f914f2, COVID: 0
INDEX 3266 - Processing UUID: fa2b79f3-d81b-477e-b6b1-f9bf7dd6e9c5, COVID: 0
INDEX 595 - Processing UUID: 40eb8781-0351-4eab-ac8f-3036986e5b37, COVID: 0


...

INDEX 1211 - Processing UUID: 51a9ade4-721c-4a75-ab33-57e68f425a42, COVID: 0


/opt/conda/lib/python3.10/site-packages/matplotlib/axes/_axes.py:7774: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


INDEX 3391 - Processing UUID: e92dbe53-38eb-4ff0-bf63-85ed6cf56a86, COVID: 0


/opt/conda/lib/python3.10/site-packages/matplotlib/axes/_axes.py:7774: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


INDEX 1677 - Processing UUID: da71535e-5a96-401c-aaa3-1eb43c5331d2, COVID: 0
Processed: 2167, Skipped: 152


In [10]:
import shutil

# Paths to the directories
#train_output_dir = '/kaggle/working/train_spectrograms/'
test_output_dir = '/kaggle/working/test_spectrograms/'

# Paths to the output ZIP files
#train_zip_path = '/kaggle/working/train_spectrograms.zip'
test_zip_path = '/kaggle/working/test_spectrograms.zip'

# Create a ZIP file for the training spectrograms
#shutil.make_archive(train_zip_path.replace('.zip', ''), 'zip', train_output_dir)

# Create a ZIP file for the testing spectrograms
shutil.make_archive(test_zip_path.replace('.zip', ''), 'zip', test_output_dir)

print("Spectrograms have been zipped.")

Spectrograms have been zipped.


In [11]:
# Plot the distribution of classes in the training dataset
#train_class_counts = pd.Series(train_labels).value_counts()
#plt.figure(figsize=(8, 6))
#plt.bar(train_class_counts.index, train_class_counts.values, color=['blue', 'orange'])
#plt.xticks(train_class_counts.index, ['No COVID', 'COVID'])
#plt.xlabel('Class')
#plt.ylabel('Count')
#plt.title('Distribution of Classes in the Training Dataset')
#plt.show()